In [1]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import json

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Using TensorFlow backend.


In [12]:
def get_text(rev):
    return rev[0] if str(rev)!='nan' else ''

def clean_text(text):
    import re, string
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub('', text)
    text = re.sub('/', ' ', text)
    text = re.sub('\s+', ' ', text)
#     text = re.sub("\'", "", text)
    return text.lower()

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
#         elif token.like_url:
#             lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def spell_correction(text):
    from textblob import TextBlob
    return TextBlob(text).correct()

def prepare_text_for_lda(text):
    tokens = tokenize(text)
#     tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package wordnet to /home/argha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/argha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
%%time
import pickle
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))


CPU times: user 729 ms, sys: 109 ms, total: 838 ms
Wall time: 847 ms


In [4]:
# %%time
import pyLDAvis.gensim
lda5 = gensim.models.ldamodel.LdaModel.load('model5.gensim')
lda_display5 = pyLDAvis.gensim.prepare(lda5, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display5)

/home/argha/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [6]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/home/argha/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [8]:
topics = lda10.print_topics(num_topics=10,num_words=5)
for topic in topics:
    print(topic)

(0, '0.111*"value" + 0.082*"like" + 0.066*"money" + 0.052*"cleanliness" + 0.043*"rooftop"')
(1, '0.312*"breakfast" + 0.051*"great" + 0.036*"include" + 0.025*"comfortable" + 0.023*"proximity"')
(2, '0.044*"check" + 0.040*"bathroom" + 0.038*"clean" + 0.034*"comfortable" + 0.023*"night"')
(3, '0.093*"square" + 0.089*"location" + 0.075*"times" + 0.074*"central" + 0.041*"station"')
(4, '0.097*"everything" + 0.048*"hotel" + 0.040*"staff" + 0.025*"need" + 0.023*"could"')
(5, '0.402*"location" + 0.190*"great" + 0.079*"excellent" + 0.073*"staff" + 0.040*"service"')
(6, '0.125*"hotel" + 0.045*"would" + 0.043*"great" + 0.041*"place" + 0.041*"amaze"')
(7, '0.205*"staff" + 0.144*"clean" + 0.139*"friendly" + 0.102*"helpful" + 0.097*"comfortable"')
(8, '0.087*"location" + 0.069*"close" + 0.050*"restaurant" + 0.039*"convenient" + 0.037*"price"')
(9, '0.052*"coffee" + 0.042*"lobby" + 0.039*"nothing" + 0.028*"hotel" + 0.021*"available"')


In [14]:
new_doc = 'I don\'t like this hotel at all. The bathrooms are filled with bugs'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
# print(new_doc_bow)
print(lda10.get_document_topics(new_doc_bow))

[(0, 0.2200739), (1, 0.019997673), (2, 0.21999864), (3, 0.019999754), (4, 0.41992593), (5, 0.01999767), (6, 0.020007493), (7, 0.019998888), (8, 0.020000326), (9, 0.01999978)]


In [15]:
nn=lda10.get_document_topics(new_doc_bow)

In [20]:
nn[0][1]

0.22007388

In [17]:
np.sort(nn[])

array([[0.        , 0.22007388],
       [0.01999767, 1.        ],
       [0.21999867, 2.        ],
       [0.01999975, 3.        ],
       [0.41992593, 4.        ],
       [0.01999767, 5.        ],
       [0.02000741, 6.        ],
       [0.01999888, 7.        ],
       [0.02000032, 8.        ],
       [0.01999978, 9.        ]])

In [16]:
# pyLDAvis.save_html(lda_display10, 'lda10.html')

In [ ]:
lda10_topic={0:'Value_for_money',1:''}